- Source: 

In [2]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Grab Data
import yfinance as yf

# Common Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# pip3 install torch torchvision torchaudio - URL https://pytorch.org/get-started/locally/
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LSTM
import torch.optim as optim
from torch.autograd import Variable

In [4]:
# Download the data
# The EUR/USD exchange rate data from Yahoo Finance
data = yf.download("EURUSD=X", start="2013-01-01", end="2025-04-11")["Close"]

# Create a DataFrame from the Downloaded Data
raw_data = pd.DataFrame(data).reset_index()
raw_data.rename(columns={ "EURUSD=X": "Close"}, inplace=True)  # Change 'Close' to 'EURUSD=X'

print(raw_data.columns)

# Calculate the returns and add them to the DataFrame
raw_data["Returns"] = raw_data["Close"].pct_change()

# When the market direction is greater than 0 --> class 1 (up), else class 0 (down)
raw_data["Direction"] = np.where(raw_data["Returns"] > 0, 1, 0)

print(raw_data.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Index(['Date', 'Close'], dtype='object', name='Ticker')
Ticker       Date     Close   Returns  Direction
0      2013-01-01  0.757748       NaN          0
1      2013-01-02  0.757926  0.000235          1
2      2013-01-03  0.758553  0.000827          1
3      2013-01-04  0.767931  0.012364          1
4      2013-01-07  0.765333 -0.003383          0
